<a href="https://colab.research.google.com/github/AymenTlili131/The-sound-of-Ai-Hack-V1.0/blob/working_proposals/Source_Separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

try:
  import google.colab
  IN_COLAB = True       
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install musdb
    !pip install openunmix
else:
    print(sys.executable)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 963 kB 11.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.4 MB/s 


In [2]:
import os, glob, shutil

if IN_COLAB:
    from google.colab import drive
    '''
    If using Google Drive, change Folder where dataset is contained
    '''
    ROOT = '/content/drive/'
    drive.mount(ROOT)
    COLAB_FOLDER = os.path.join(ROOT, 'My Drive/Hack v1.0/')
    print('Colab project folder: ', COLAB_FOLDER)
    COLAB_DATA = os.path.join(COLAB_FOLDER, 'Dataset/')
    print('Colab dataset folder: ', COLAB_DATA)
else:
    '''
    If running locally, set where datasets are contained. Annotations and audio folders have to be described as such.
    '''
    DATA_FOLDER = ''
    print('Dataset folder: ', DATA_FOLDER)

Mounted at /content/drive/
Colab project folder:  /content/drive/My Drive/Hack v1.0/
Colab dataset folder:  /content/drive/My Drive/Hack v1.0/Dataset/


In [3]:
import torch
import torchaudio
import numpy as np
import scipy
import stempeg
import librosa
import re
import openunmix
from tqdm.notebook import tqdm
from time import sleep
from scipy.io import wavfile
from IPython.display import Audio, display

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device\n\n")
if device == "cuda":
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    print(gpu_info)

# Enable pushing tensors to GPU for speeding up training
if device == "cuda": 
    pin_mem = False
    n_workers = 0
else: 
    pin_mem = False
    n_workers = 0

# Set number of CPU threads
n_threads = 2
torch.set_num_threads(n_threads)

print('pin_memory: ', pin_mem)
print('Number of available CPU\'s: ', os.cpu_count())
print('Number of CPU threads: ', torch.get_num_threads())

Using cuda device


Fri Jul  8 21:41:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                    

In [5]:
print('Current working directory: ', os.getcwd())
print(COLAB_DATA)

# Control with a list which datasets to import into Colab, avoiding cluttering
DATASETS = ['1000_songs']

# Set for deleting past datasets and reload datasets from list
RELOAD_DATASETS = False

def import_datasets(data_folder, dataset_list):
    print('Importing datasets')
    for root, dirs, files in os.walk(data_folder):
        for name in dirs:
            if name in dataset_list:
                print(name)
                if not os.path.exists('./dataset/' + name):
                    data_audio = './dataset/' + name + '/Audio'
                    os.makedirs(data_audio, exist_ok=True)
                    for file_ in tqdm(os.listdir(os.path.join(root, name)), desc=f'dataset: {name}'):
                        shutil.copy2(os.path.join(root, name, file_), data_audio)
                    print('Dataset folder created: ', './dataset/' + name)

if IN_COLAB:
    if RELOAD_DATASETS:
        # Delete folders for fresh start (newly added data to drive, etc.)
        !rm -rf ./dataset
        !rm -rf ./output
    os.makedirs('./dataset', exist_ok=True)
    os.makedirs('./output', exist_ok=True)
    import_datasets(COLAB_DATA, DATASETS)

else:
    '''
    Type in the folder directories where the .txt annotations and .wav audio
    are contained
    '''    
    if RELOAD_DATASETS:
        shutil.rmtree('./dataset')
        shutil.rmtree('./output')
    os.makedirs('./dataset', exist_ok=True)
    os.makedirs('./output', exist_ok=True)
    
    import_datasets(DATA_FOLDER, DATASETS)

Current working directory:  /content
/content/drive/My Drive/Hack v1.0/Dataset/
Importing datasets
1000_songs


dataset: 1000_songs:   0%|          | 0/50 [00:00<?, ?it/s]

Dataset folder created:  ./dataset/1000_songs


In [6]:
DIR = 'dataset/1000_songs/Audio/'
filenames = os.listdir(DIR)
filenames = sorted(filenames)
filenames = filenames[:3]
print(filenames)
print(len(filenames))

['32Stitches - Olympus [NCS Release].wav', '3rd Prototype & Emdi - House [NCS Release].wav', "3rd Prototype - I'm Fine [NCS Release].wav"]
3


In [37]:
# from openunmix import predict
# audio, sr = librosa.load(os.path.join(DIR, '32Stitches - Olympus [NCS Release].wav'), sr=44100, mono=False)
# estimates = predict.separate(audio=torch.as_tensor(audio.T).float(), rate=sr,
#                                 model_str_or_path='umxhq', 
#                                 targets=['vocals'],
#                             #  targets=['vocals','drums', 'bass','other'], 
#                                 residual=True, 
#                                 device=device,
#                                 )

Downloading: "https://zenodo.org/api/files/1c8f83c5-33a5-4f59-b109-721fdd234875/vocals-b62c91ce.pth" to /root/.cache/torch/hub/checkpoints/vocals-b62c91ce.pth


  0%|          | 0.00/34.0M [00:00<?, ?B/s]

In [13]:
from openunmix import predict

def normalize_32float_to_16int(waveform):
    waveform_norm = 2 * (waveform - np.amin(waveform)) / (np.amax(waveform) - np.amin(waveform)) - 1
    waveform_norm *= 32767
    waveform_norm = waveform_norm.astype(np.int16)
    return waveform_norm

for name in tqdm(filenames):
    audio, sr = librosa.load(os.path.join(DIR, name), sr=44100, mono=False)
    estimates = openunmix.predict.separate(audio=torch.as_tensor(audio.T).float(), rate=sr,
                                 model_str_or_path='umxhq', 
                                 targets=['vocals'],
                                #  targets=['vocals','drums', 'bass','other'], 
                                 residual=True, 
                                 device=device,
                                 )

    vocals = estimates['vocals'].squeeze().detach().cpu().numpy()
    residual = estimates['residual'].squeeze().detach().cpu().numpy()
    
    # Avoid running out of memory, clear GPU
    del estimates
    torch.cuda.empty_cache()

    vocals_16int = normalize_32float_to_16int(vocals)
    residual_16int = normalize_32float_to_16int(residual)

    # print('\nVocals max/min array values')
    # print(vocals_16int.dtype)
    # print(np.amax(vocals_16int))
    # print(np.amin(vocals_16int))
    # print('\nResidual max/min array values')
    # print(residual_16int.dtype)
    # print(np.amax(residual_16int))
    # print(np.amin(residual_16int))


    # new_name_vocals = name.replace('.wav', '') + '_vocals.wav'
    # new_name_residual = name.replace('.wav', '') + '_residual.wav'
    new_name_vocals = name.replace('.wav', '') + '_vocals_16int.wav'
    new_name_residual = name.replace('.wav', '') + '_residual_16int.wav'

    wavfile.write('./output/' + new_name_vocals, rate=44100, data=vocals_16int.T) # Shape must be [samples, channel]
    wavfile.write('./output/' + new_name_residual, rate=44100, data=residual_16int.T) # Shape must be [samples, channel]
    # wavfile.write('/content/drive/MyDrive/Hack V1.0/' + new_name_vocals, rate=44100, data=vocals.T) # Shape must be [samples, channel]
    # wavfile.write('/content/drive/MyDrive/Hack V1.0/' + new_name_residual, rate=44100, data=residual.T) # Shape must be [samples, channel]

  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
# Listen to  single example (has to be shortend or iPython collapses)

def play_sample(file_loc: str, duration: float):
    sr, audio = wavfile.read(file_loc)
    frames = int(duration * sr)
    audio = audio[:frames, :]
    return Audio(data=audio.T, rate=sr)

In [ ]:
play_sample('/content/output/32Stitches - Olympus [NCS Release]_residual_16int.wav', duration=15.0)

In [28]:
play_sample('/content/output/32Stitches - Olympus [NCS Release]_vocals_16int.wav', duration=15.0)

In [ ]:
# Download files locally (if needed)
google.colab.files.download('/content/output.zip')